**Ejercicio 3:** Pruebe varios valores de k(por ejemplo entre 2 y 10) para el conjunto de datos Iris, calcule para cada k una métrica de clustering (con sklearn) y encuentre el k óptimo para k-medias.

#### <b>Librerías</b>

In [1]:
import random
import csv
import numpy as np
from tabulate import tabulate

from kmedias_func import kmedias

from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.metrics.cluster import davies_bouldin_score

#### **Golden Standard**

In [2]:
archivo = './data/irisbin_trn.csv'
trn = np.loadtxt(archivo,delimiter=',')
cant_e = 4
yd = np.empty(len(trn),dtype=object)         # Salidas esperadas

for i in range(len(trn)):
    fila = trn[i]
    yd[i] = fila[cant_e:]

idx_yd = [np.argmax(v) for v in yd]         # Codifico las salidas para comparar

#### **Métricas de clustering para k-medias con distintos k**

In [3]:
matrix = []
ri = []
fms = []
db = []

for k in range(2,11):
    inputs,input_centroid = kmedias(k,cant_e,archivo)

    matrix.append(contingency_matrix(idx_yd,input_centroid))
    ri.append(rand_score(idx_yd,input_centroid))
    fms.append(fowlkes_mallows_score(idx_yd,input_centroid))
    db.append(davies_bouldin_score(np.vstack(inputs),input_centroid))


Reuno los datos en una tabla. Recordemos:
- **Matriz de contingencia:** Me muestra coincidencias entre los índices.
- **Rand Index:** Mientras más cerca de 1, mejor clustering.
- **Fowlkes & Mallows:** Mientras más cerca de 1, mejor clustering.
- **Medida Combinada:** Mínimo cero, menor valor implica mejor clustering.

In [4]:
table_data = [[x,y,z,k,l] for x, y, z, k, l in zip(range(2,11), matrix, ri, fms, db)]
headers = ['k','Matriz de Contingencia','Rand Index','Fowlkes & Mallows','Medida Combinada']
table = tabulate(table_data, headers, tablefmt='simple_grid',stralign='center',numalign='center')
print(table)

┌─────┬───────────────────────────────────┬──────────────┬─────────────────────┬────────────────────┐
│  k  │      Matriz de Contingencia       │  Rand Index  │  Fowlkes & Mallows  │  Medida Combinada  │
├─────┼───────────────────────────────────┼──────────────┼─────────────────────┼────────────────────┤
│  2  │             [[ 0 34]              │   0.821785   │      0.808054       │      0.375535      │
│     │              [ 0 32]              │              │                     │                    │
│     │              [45  0]]             │              │                     │                    │
├─────┼───────────────────────────────────┼──────────────┼─────────────────────┼────────────────────┤
│  3  │            [[ 8  0 26]            │   0.893857   │       0.84203       │      0.735411      │
│     │             [28  0  4]            │              │                     │                    │
│     │            [ 0 45  0]]            │              │                     │  

C:\Users\ssfes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tabulate\__init__.py:108: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  or (len(row) >= 2 and row[1] == SEPARATING_LINE)
